In [1]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!pip install webdriver_manager

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
!pip install bs4 lxml

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
!pip install pymongo

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

In [6]:
from bs4 import BeautifulSoup

In [7]:
driver = webdriver.Chrome(ChromeDriverManager().install())

/tmp/ipykernel_44516/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [8]:
driver.get("https://edugate.ksu.edu.sa/ksu/ui/guest/timetable/index/mainScheduleTreeCoursesIndex.faces")

In [9]:
driver.find_element(By.ID,'myForm:timetableTreeId').click()

In [10]:
# driver.find_element_by_id('myForm:timetableTreeId').click()

In [11]:
def has_setIndex(tag):
    return tag.has_attr('href') and 'setIndex' in tag['href']

In [12]:
source = driver.page_source
sp = BeautifulSoup(source)
tree = sp.find('div',{"class":"dtree"})

majors_dict = {}
for i in tree.find_all(has_setIndex):
    start = i['href'].find("'") + 1
    end = i['href'].rfind("'")
    majors_dict[int(i['href'][start:end])] = i.text 

In [13]:
print("%5s" % "id:","Major Name")
for i, major in majors_dict.items():
    print("%5d" %i, ':', major)

  id: Major Name
    0 : MRKN-التسويق
   10 : IMNN-المالية
   11 : IMN1-مسار المالية العام
   20 : EC1-الاقتصاد العام
   21 : ECN1-مسار الاقتصاد العام
   22 : ECN2-مسار المصرفية الاسلامية
   30 : PUAN-الإدارة
   31 : PUN1-مسار الإدارة
   32 : PUN2-مسار إدارة الموارد البشرية
   40 : UMP01-إدارة الأعمال عام
   50 : ACTN-المحاسبة
   60 : INFN-نظم المعلومات الإدارية
  100 : UPARB-لغة عربية
  110 : MCPR-إعلام-علاقات عامه
  111 : UPMC-إعلام (عام)
  112 : AVM-مسار الإعلام المرئي والمسموع
  113 : PPM-مسار الصحافة والنشر الإلكتروني
  114 : SCMX-مسار العلاقات العامة
  120 : UPHIS-تاريخ
  130 : UPGEO-جغرافيا
  131 : GOGD-مسار التنمية الإقليمية
  132 : GOGM-مسار الخرائط ونظم المعلومات الجغرافية
  140 : UPSC-علم الإجتماع
  141 : UPPS-الخدمة الاجتماعية
  150 : UPLIB-علم المعلومات
  151 : LISM-مسار إدارة مصادر المعلومات وخدماتها
  152 : LRME-مسار إدارة السجلات والحفظ الإلكتروني
  160 : UPENG-اللغة الإنجليزية
  200 : UPPO-العلوم السياسية
  210 : UPLAW-الحقوق
  300 : ICBL-دراسات اسلاميه (العقيدة)
  301

In [14]:
def select_major(mId):
    x = str(mId)
    driver.execute_script(f"setIndex('{x[:2] if len(x) > 2 else '0'}_{x[-2] if len(x) > 1 else '0'}_{x[-1]}')")

In [15]:
mId = int(input("Enter major id from the above table:"))
if mId not in majors_dict:
    print("Wrong ID, try again!")

Enter major id from the above table:1500


In [16]:
select_major(mId)

In [17]:
driver.find_element(By.XPATH,'//*[@id="myForm"]/table[4]/tbody/tr[1]/td/table/tbody/tr/td/table/tbody/tr[5]/td[6]/a[2]/table/tbody/tr/td[1]/img').click()

In [18]:
content = driver.page_source
soup = BeautifulSoup(content)
rt= soup.find(id="myForm:timetable")
# rt

In [19]:
trs = rt.find_all('tr')
print(len(trs[1:]))

32


In [20]:
def press_details(index=1):
    details_selector = '#myForm\:timetable > tbody > tr:nth-child({}) > td:nth-child(8) > a'
    details_button = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,details_selector.format(index))))
    details_button.click()

In [21]:
def press_details_js(index=0):
    driver.execute_script(f"showToolTipDilaog(event,'{index}');")

In [22]:
def get_times():
#     times = driver.find_element_by_id('sectionDiv').find_elements_by_tag_name('nobr')
    times = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.ID,'sectionDiv'))).find_elements(By.TAG_NAME,'nobr')
    if not times:
        return None
    n=len(times)//3
    return [times[i*3].text for i in range(n)], [times[1+i*3].text for i in range(n)] , [times[2+i*3].text for i in range(n)]

In [23]:
def get_final_info():
    return driver.find_element(By.ID,'examDayDiv').text,driver.find_element(By.ID,'examTimeDiv').text,driver.find_element(By.ID,'examDateDiv').text

In [24]:
def get_instructor():
    return driver.find_element(By.ID,'instructorDiv').text

In [25]:
def close():
    driver.find_element(By.XPATH,'//*[@id="dlg"]/div[1]/a/span').click()

In [26]:
ctrs = trs[1:]
len(ctrs)

32

In [27]:
# NOTE: the following method does not collect courses where the activity is "تدريب" or "مشروع".
# If you want to include thenm
res = []
for x, i in enumerate(ctrs):
    tds = i.find_all('td')
    if 'مشروع' in tds[3].text or 'تدريب' in tds[3].text: continue
    tmp = {
        'cid': tds[0].text,
        'cname': tds[1].text,
        'section': tds[2].text,
        'activity': tds[3].text,
        'chours': tds[4].text,
        'gender': tds[5].text,
        'status': tds[6].text
    }
    press_details_js(x)
    driver.implicitly_wait(5)
    times = get_times()
    tmp['days'],tmp['time'],tmp['location']=(times) if times else ('','','')
    final_info = get_final_info()
    tmp['fexamday'],tmp['fexamtime'],tmp['fexamdate']=final_info

    tmp['instructor'] = get_instructor()
    driver.implicitly_wait(5)
    close()
    driver.implicitly_wait(5)
    res.append(tmp)

In [28]:
# NOTE: the following method does not collect courses where the activity is "تدريب" or "مشروع".
# If you want to include thenm
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")
db = client["edugate_courses"]
collection = db[f'{majors_dict[mId]}']
collection.insert_many(res)
# for x, i in enumerate(ctrs):
#     tds = i.find_all('td')
#     if 'مشروع' in tds[3].text or 'تدريب' in tds[3].text: continue
#     tmp = {
#         'cid': tds[0].text,
#         'cname': tds[1].text,
#         'section': tds[2].text,
#         'activity': tds[3].text,
#         'chours': tds[4].text,
#         'gender': tds[5].text,
#         'status': tds[6].text
#     }
#     press_details_js(x)
#     driver.implicitly_wait(5)
#     times = get_times()
#     tmp['days'],tmp['time'],tmp['location']=(times) if times else ('','','')
#     final_info = get_final_info()
#     tmp['fexamday'],tmp['fexamtime'],tmp['fexamdate']=final_info

#     tmp['instructor'] = get_instructor()
#     driver.implicitly_wait(5)
#     close()
#     driver.implicitly_wait(5)
#     collection.insert_one(tmp)

In [29]:
driver.quit()

In [30]:
len(res)

30

In [31]:
import csv
keys = res[0].keys()

with open(f'{majors_dict[mId]}.csv', 'w',encoding='utf-8-sig', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(res)

In [32]:
import json
with open(f'{majors_dict[mId]}.json', 'w', encoding="utf-8-sig")as output_file:
    json.dump(res,output_file)

TypeError: Object of type ObjectId is not JSON serializable